## <span style="color:blue">G112 Summary</span>

### <span style="color:#008891">Incidence of industrial pellets by location, amplitude and frequency</span>

Plastic industrial pellets or GPI have been found at all lakes in the present study. The quantity of pellets found and the frequency at which they are identified varies from one lake to another and from one location to another.

Here we summarize the incidence of GPI nationally and regionally. We focus on the locations that have the geatest median value for GPI and the greatest frequency.

#### <span style="color:#008891">Discussion</span>

questions or comments: analyst@hammerdirt.ch

In [41]:
# sys, file and nav packages:
import os
import datetime as dt
import csv

# math packages:
import pandas as pd
import numpy as np
from scipy import stats
import datetime as dt 


# charting:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import ticker
import seaborn as sns
import matplotlib.gridspec as gridspec
from IPython.display import display, HTML


# home brew utitilties
import utilities.utility_functions as ut

# documenting
from IPython.display import Markdown as md

display(HTML("""
<style>
.output {
    display: flex;
    align-items: center;
    text-align: center;
}
</style>
"""))

In [42]:
# the local file structure. The resources are located in the corresponding directory.
survey_data, location_data, code_defs, stat_ent, geo_data, output = ut.make_local_paths()

In [43]:
# variables/arrays that are frequently used:
# project lakes
the_lakes = [
    "Bielersee",
    "Walensee",
    "Lac Léman",
    "Zurichsee",
    "Neuenburgersee",
    "Thunersee",
    "Lago Maggiore",
    "Brienzersee",
    "poi"
]

the_rivs = [
    'Aare',
    'Rhône',
    'Limmat',
    'Linth',
    'Ticino',
]
# project beaches
poi = [
    "neuenburgersee_cudrefin_mattera",
    "la-petite-plage",
    "evole-plage",
    "plage-de-cheyres",
    "mullermatte",
    "bielersee_vinelz_fankhausers",
    "baby-plage-geneva",
    "baye-de-montreux-g",
    "grand-clos",
    "preverenges",
    "quai-maria-belgia",
    "zurichsee_wollishofen_langendorfm",
    "zurichsee_kusnachterhorn_thirkell-whitej",
    "zuerichsee_richterswil_benkoem_2",     
    "rastplatz-stampf",
    "weissenau-neuhaus",
    "thunersee_spiez_meierd_1",
    "walensee_walenstadt_wysse",
    "gasi-strand",
    "vira-gambarogno",
    "sentiero-giro-del-golf-spiaggia",
    "maladaire"
    
]

In [44]:
# standard formats already in use for charts 
# you can just apply these as kwargs to different elements...
table_k = dict(loc="top left", bbox=(0,0,1,1), colWidths=[.5, .5], cellLoc='center')
tablecenter_k = dict(loc="top left", bbox=(0,0,1,1), cellLoc='center')
tabtickp_k = dict(axis='both', which='both', bottom=False, top=False, left=False, right=False, labelleft=False, labelbottom=False)
title_k = {'loc':'left', 'pad':10, 'linespacing':1.5, 'fontsize':12}
title_k20 = {'loc':'left', 'pad':10, 'linespacing':1.5, 'fontsize':12, 'color':'dodgerblue'}
title_k17 = {'loc':'left', 'pad':10, 'linespacing':1.5, 'fontsize':12, 'color':'salmon'}
titler_k20 = {'loc':'right', 'pad':10, 'linespacing':1.5, 'fontsize':12, 'color':'dodgerblue'}
titler_k17 = {'loc':'right', 'pad':10, 'linespacing':1.5, 'fontsize':12, 'color':'salmon'}
xlab_k = {'labelpad':10, 'fontsize':12}
ylab_k = {'labelpad':14, 'fontsize':14}
titler_k = {'loc':'right', 'pad':10, 'linespacing':1.5, 'fontsize':12}

# use thes to format date axis in charts
weeks = mdates.WeekdayLocator(byweekday=1, interval=4)
days = mdates.DayLocator(bymonthday=1, interval=1)
months = mdates.MonthLocator(bymonth=[3,6,9,12])
wks_fmt = mdates.DateFormatter('%d')
mths_fmt = mdates.DateFormatter('%b')

In [45]:
# probably want to keep these... the works already done
# aggregated survey data
dfAgg = pd.read_csv(F"{survey_data}/results_with_zeroes_aggregated_parent.csv")
dfAgg = dfAgg[dfAgg.location != 'signalpain']
dfAgg['date'] = pd.to_datetime(dfAgg['date'])

# non aggregated survey data
dfSurveys = pd.read_csv(F"{survey_data}/results_with_zeroes.csv")
dfSurveys['date'] = pd.to_datetime(dfSurveys['date'])

# beach data
dfBeaches = pd.read_csv(F"{location_data}/beaches_pop_bfs.csv")
dfSurveys = dfSurveys[dfSurveys.location != 'signalpain']
dfBeaches = dfBeaches[dfBeaches.slug != 'signalpain']
dfBeaches.set_index('slug', inplace=True)

# code definitions
dfCodes = pd.read_csv(F"{code_defs}/mlw_codes.csv")

# geo date: explantory variables, index by slug and make a map:
dfStreets = pd.read_csv(F"{geo_data}/exp_variables/strts_1000m.csv", index_col='slug')['length']
dfBlds = pd.read_csv(F"{geo_data}/exp_variables/bldgs_500m.csv", index_col='slug')['area']
dfRivs = pd.read_csv(F"{geo_data}/exp_variables/rivs_1500m.csv", index_col='slug')['NUMPOINTS']

#### <span style="color:#008891">Location of project folder</span>

In [46]:
g112summary = ut.make_project_folder(output, 'g112Summary')
md(F"### The project directory is: ../{g112summary}")

That project exists


### The project directory is: ../output/g112Summary

### <span style="color:#008891">Industrial pellets: summary</span>

1. Locations where they have been found
2. Frequency of identification
3. Seasonal variation
4. Correlation tables
5. Median value per region

In [37]:
# set some parameters:
start_date = '2020-04-01'
end_date = dt.datetime.today().strftime('%Y-%m-%d')
code = 'G112'

# restrict to lakes only
dfS = dfAgg.loc[(dfAgg.water_name.isin(the_lakes))&(dfAgg.date >= start_date)].copy()

# map geo values to survey results:
dfS['population']=dfS.location.map(lambda x: dfBeaches.loc[x]['population'])
dfS['streets'] = dfS.location.map(lambda x: dfStreets.loc[x])
dfS['buildings'] = dfS.location.map(lambda x: dfBlds.loc[x])
dfS['rivs'] = dfS.location.map(lambda x: dfRivs.loc[x])

# seperate out the codes of interest
dfCode = dfS.loc[dfS.code == code]

# daily totals for each group, index to date
allDf = dfS.groupby(['loc_date', 'date'], as_index=False).pcs_m.sum()
allDf.set_index('date', inplace=True)

g112Df = dfCode.groupby(['loc_date', 'date'], as_index=False).pcs_m.sum()
g112Df.set_index('date', inplace=True)

In [ ]:
# save some data
# cone.to_csv(F"{g112summary}/group_one_agg.csv", index = False)

#### Hopefully that just worked for you

if not contact analyst@hammerdirt.ch